
<h1><font color=red>Building a Recommender using AutoMLx</font></h1>
<p style="margin-left:5%; margin-right:5%;">by the <font color=teal> Oracle AutoMLx Team </font></p>

***

Recommendation Demo Notebook.

Copyright © 2024, Oracle and/or its affiliates.

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

# Overview of this Notebook

In this notebook we will build a recommender using the Oracle AutoMLx tool for the Movielens 100k dataset to predict the next item that users will most likely watch, based on their ratings history.
We explore the various options provided by the Oracle AutoMLx tool, allowing the user to control the AutoMLx training process. We finally evaluate the different models trained by AutoMLx. Depending on the machine running this notebook, it can take up to minutes.

---
## Prerequisites:

   - Experience level: Novice (Python and Machine Learning)
   - Professional experience: Some industry experience
---

## Business Use:

Data analytics and modeling problems using Machine Learning (ML) are becoming popular and often rely on data science expertise to build accurate ML models. Such modeling tasks primarily involve the following steps:
- Preprocess dataset (clean, impute, engineer features, normalize).
- Pick an appropriate model for the given dataset and prediction task at hand.
- Tune the chosen model’s hyperparameters for the given dataset.

All of these steps are significantly time consuming and heavily rely on data scientist expertise. Unfortunately, to make this problem harder, the best feature subset, model, and hyperparameter choice widely varies with the dataset and the prediction task. Hence, there is no one-size-fits-all solution to achieve reasonably good model performance. Using a simple Python API, AutoML can quickly jump-start the datascience process with an accurately-tuned model and appropriate features for a given prediction task.

## Table of Contents

- <a href='#setup'> Setup</a>
- <a href='#load-data'> Load the Movielens 100k dataset</a>
  - <a href='#coltypes'> Define the column types</a>
  - <a href='#dataset-split'>Splitting the dataset</a>
- <a href='#automl'> AutoML</a>
  - <a href='#provider'> Create an Instance of AutoMLx</a>
  - <a href='#default'> Train a Model using AutoMLx</a>
  - <a href='#recommending'> Generate recommendations </a>
  - <a href='#analyze'> Analyze the AutoMLx optimization process </a>
      - <a href='#algorithm-selection'> Algorithm Selection</a>
      - <a href='#hyperparameter-tuning'> Hyperparameter Tuning</a>
  - <a href='#modellist'> Advanced AutoMLx Configuration</a>
  - <a href='#custom-validation'> Use a custom validation set</a>
  - <a href='#evaluation'> Final evaluation of the best model</a>

<a id='setup'></a>
## Setup

Basic setup for the Notebook.

In [ ]:


import datetime
import logging
import os
import time
import urllib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from automlx import AutoRecommender, init

# Settings for plots
plt.rcParams["figure.figsize"] = [10, 7]
plt.rcParams["font.size"] = 15

# Silence unnecessary warnings
logging.getLogger("sanerec.autotuning.parameter").setLevel(logging.ERROR)

# Initialize the parallelization engine of AutoMLx
init(engine='ray', engine_opts={"ray_setup": {"log_to_driver": False}})

<a id='load-data'></a>
## Load Movielens 100k data
Movielens 100k dataset is one of the most common public datasets for movie recommendation. It contains 100k ratings from about 1k users on 1.6k movies, some information about user demographic, and additional movie characteristics. For more information about this dataset, you can visit the [Movielens website](https://grouplens.org/datasets/movielens/100k/).

In this demo, we use the ratings to train a movie recommendation model, exploiting AutoMLx to find the best recommendation model and hyperparameters to use in terms of recommendation accuracy.
Therefore, we start retrieving and loading the ratings data of the Movielens 100k dataset.

In [ ]:


get_ipython().system(' wget https://files.grouplens.org/datasets/movielens/ml-100k/u.data --no-check-certificate -q -O ./ml100k_interactions.tsv')

In [ ]:


dataset = pd.read_csv(
    "./ml100k_interactions.tsv",
    sep="\t",
    names=["user_id", "movie_id", "rating", "timestamp"],
)

dataset.head(5)

In order to be used for the recommendation task, the data must have a timestamp column that is used to infer the temporal order of the samples. We also require to set the timestamp column as index of the dataframes used in our AutoML pipelines.

Movielens contains a `timestamp` column that contains the time when a rating was given, so we set it as index of our dataframe.

In [ ]:


dataset = dataset.set_index("timestamp")
dataset.head(5)

<a id='coltypes'></a>
### Define types of columns in the dataframe

The recommendation task requires to define the two main entities involved in the recommendation:
- the `recommendation`, which represents the entity type that is going to be recommended;
- the `recommendation_subject`, which represents the entity type that receives the recommendation.

For this reason, AutoML requires to indicate what are the columns in the dataset that refer to these two concepts, and, in particular, the two columns that contain their unique identifiers.

In our demo we want to recommend movies (`recommendation`), identified by the `movie_id` column, to users (`recommendation_subject`), identified by the `user_id` column. We declare this binding in a python dictionary that we will reuse throughout the demo.

In [ ]:


col_types = {"movie_id": "recommendation", "user_id": "recommendation_subject"}

<a id='dataset-split'></a>
## Splitting the dataset

We split the dataset into training and test datasets using a leave-last-out technique.
The training set will be used to create a Machine Learning model using Oracle AutoMLx, and the test set will be used to evaluate the model's performance on unseen data.

The leave-last-out splitting technique consists in keeping in the test set only the last data sample, as determined by its timestamp, for each `recommendation_subject` (user in this case). All the other samples form the training set. This corresponds to the common next item recommendation use case, where given the history of all the past data concerning a `recommendation_subject` in the training set, we want to predict what should be recommended next to the same subject, and check if it corresponds to the actual sample in the test set.

In [ ]:


training_data, test_data = AutoRecommender.train_test_split(data=dataset, col_types=col_types)

<a id='automl'></a>
# AutoML

<a id='provider'></a>
## Create an instance of Oracle AutoMLx

The Oracle AutoMLx solution provides a pipeline that automatically finds a tuned model given a prediction task and a training dataset. In particular, it allows finding a tuned model for any supervised prediction task, for example, classification or regression where the target can be binary, categorical or real-valued.

In this demo we want a model that performs a recommendation task, so we create a pipeline of type `AutoRecommender`, and we configure it with default parameters. You can find the complete list of all the available parameters and their meaning in our documentation.

In [ ]:


automl_pipeline = AutoRecommender().configure()

<a id='default'></a>
## Train a model using AutoMLx

The training data is passed to the `fit()` function which executes the model selection and hyperparameter tuning steps.

In [ ]:


automl_pipeline = automl_pipeline.fit(data=training_data, col_types=col_types)

<a id='recommending'></a>
## Generate recommendations

Once the AutoML pipeline is completed, we predict 5 recommendations for a random user in the dataset.

In [ ]:


recommendation_subjects = test_data.sample(1)[['user_id']]
automl_pipeline.predict(subjects=recommendation_subjects, n_recommendations=5)

<a id='analyze'></a>
## Analyze the AutoMLx optimization process

During the Oracle AutoMLx process for recommendation, a summary of the optimization process is logged, containing:
- Information about the training data.
- Information about the AutoMLx Pipeline, such as:
    - Selected algorithm that was the best choice for this data;
    - Selected hyperparameters for the selected algorithm.

AutoMLx provides a `print_summary` API to output all the different trials performed.

In [ ]:


automl_pipeline.print_summary()

We also provide the capability to visualize the results of each stage of the AutoMLx pipeline.

<a id='algorithm-selection'></a>
### Algorithm Selection

The plot below shows the scores predicted by Algorithm Selection for each algorithm. The horizontal line shows the average score across all algorithms. Algorithms below the line are colored turquoise, whereas those with a score higher than the mean are colored teal. The selected algorithm is in orange.

In [ ]:


def plot_model_selection_scores(_pipeline):
    # Each trial is a row in a dataframe that contains
    # Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
    trials = _pipeline.completed_trials_summary_[
        _pipeline.completed_trials_summary_["Step"].str.contains("Model Selection")
    ]
    name_of_score_column = f"Score ({_pipeline._inferred_score_metric[0].name})"
    trials.replace([np.inf, -np.inf], np.nan, inplace=True)
    trials.dropna(subset=[name_of_score_column], inplace=True)
    scores = trials[name_of_score_column].tolist()
    models = trials["Algorithm"].tolist()
    
    y_margin = 0.10 * (max(scores) - min(scores))
    s = pd.Series(scores, index=models).sort_values(ascending=False)
    
    colors = []
    for f in s.keys():
        if f.strip() == _pipeline.selected_model_.strip():
            colors.append("orange")
        elif s[f] >= s.mean():
            colors.append("teal")
        else:
            colors.append("turquoise")
    
    fig, ax = plt.subplots(1)
    ax.set_title("Algorithm Selection Trials")
    ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
    ax.set_ylabel("Hit Rate")
    s.plot.bar(ax=ax, color=colors, edgecolor="black")
    ax.axhline(y=s.mean(), color="black", linewidth=0.5)
    plt.show()

plot_model_selection_scores(automl_pipeline)

<a id='hyperparameter-tuning'></a>
### Hyperparameter Tuning

Hyperparameter Tuning is the last stage of the AutoMLx pipeline, and focuses on improving the chosen algorithm's score on the dataset. We use a novel iterative algorithm to search across many hyperparameter dimensions, and converge automatically when optimal hyperparameters are identified. Each trial represents a particular hyperparameter configuration for the selected model.


In [ ]:


def plot_hp_tuning_scores(_pipeline):
    # Each trial is a row in a dataframe that contains
    # Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
    trials = _pipeline.completed_trials_summary_[
        _pipeline.completed_trials_summary_["Step"].str.contains("Model Tuning")
    ]
    name_of_score_column = f"Score ({_pipeline._inferred_score_metric[0].name})"
    trials.replace([np.inf, -np.inf], np.nan, inplace=True)
    trials.dropna(subset=[name_of_score_column], inplace=True)
    trials.drop(trials[trials["Finished"] == -1].index, inplace=True)
    trials["Finished"] = trials["Finished"].apply(
        lambda x: time.mktime(datetime.datetime.strptime(x, "%a %b %d %H:%M:%S %Y").timetuple())
    )
    trials.sort_values(by=["Finished"], ascending=True, inplace=True)
    scores = trials[name_of_score_column].tolist()
    score = []
    score.append(scores[0])
    for i in range(1, len(scores)):
        if scores[i] >= score[i - 1]:
            score.append(scores[i])
        else:
            score.append(score[i - 1])
    y_margin = 0.10 * (max(score) - min(score))
    fig, ax = plt.subplots(1)
    ax.set_title("Hyperparameter Tuning Trials")
    ax.set_xlabel("Iteration $n$")
    ax.set_ylabel("Hit Rate")
    ax.grid(color="g", linestyle="-", linewidth=0.1)
    ax.set_ylim(min(score) - y_margin, max(score) + y_margin)
    ax.plot(range(1, len(trials) + 1), score, "k:", marker="s", color="teal", markersize=3)
    plt.show()

plot_hp_tuning_scores(automl_pipeline)

<a id='modellist'></a>
## Advanced AutoMLx Configuration

You can also configure the AutoRecommender pipeline with suitable parameters according to your needs.

In [ ]:


custom_pipeline = AutoRecommender().configure(
    model_list=[  # Specify the models you want the AutoMLx to consider
        "ItemKNNRecommender",
        "AlsRecommender",
        "BprRecommender",
    ],
    n_algos_tuned=2,  # Choose how many models to tune
    search_space={  # You can specify the hyperparameters and ranges we search for each model
        "ItemKNNRecommender": {"num_of_neighbors": {"range": [10, 30], "type": "continuous"}}
    },
    max_tuning_trials=20,  # The maximum number of tuning trials. Can be integer or Dict (max number for each model)
    score_metric="recall",  # Any of the metrics available, see the documentation for a list of supported values
)

<a id='custom-validation'></a>
## Use a custom validation set

You can specify a custom validation set that you want AutoMLx to use to evaluate the quality of models and configurations.

In [ ]:


training_data, validation_data = AutoRecommender.train_test_split(data=training_data, col_types=col_types)


# We run again the AutoML pipeline with the custom training/validation split we just created, and some advanced settings that we can specify directly in the fit method.

In [ ]:


custom_pipeline = custom_pipeline.fit(
    training_data,
    col_types,
    validation_data,
    time_budget=30,  # Specify time budget in seconds
)

Now that the custom AutoML pipeline is completed, we can generate recommendations.
Note that the pipeline's `recommend` method is equivalent to `predict`.

In [ ]:


custom_pipeline.recommend(subjects=recommendation_subjects, n_recommendations=5)

<a id='evaluation'></a>
## Final evaluation of the best model

Finally, we evaluate the best model found on the test data we have. If no metric is specified, the pipeline computes the score using the same metric used to run the Hyperparameter Tuning, which in this case is the Recall, as we defined at pipeline creation.

In this example, instead, we ask the pipeline to perform the evaluation using Normalized Discounted Cumulative Gain (NDCG), a common ranking metric. Our online documentation provides the list of the available metrics and how they are computed.

In [ ]:


get_ipython().run_line_magic('precision', '4')
custom_pipeline.score(data=test_data, score_metric="ndcg")